In [1]:
import pandas as pd

data = pd.read_csv('housing.csv')

data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
data['ocean_proximity'].unique()

In [2]:
# Crear un diccionario para mapear las categorías a valores numéricos
mapping = {
    'NEAR BAY': 0,
    '<1H OCEAN': 1,
    'INLAND': 2,
    'NEAR OCEAN': 3,
    'ISLAND': 4
}

# Aplicar el mapeo a la columna 'ocean_proximity'
data['ocean_proximity'] = data['ocean_proximity'].map(mapping)

# Verificar la conversión
print(data['ocean_proximity'].head())

# Asegurarse de que la columna es de tipo int
data['ocean_proximity'] = data['ocean_proximity'].astype(int)

0    0
1    0
2    0
3    0
4    0
Name: ocean_proximity, dtype: int64


In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data['total_bedrooms'].fillna(data['total_bedrooms'].median(), inplace=True)
data.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

# Detectar automáticamente las columnas del dataset que no son la variable dependiente
dependent_var = 'median_house_value'  # Cambiar dependiendo del dataset
independent_vars = [col for col in data.columns if col != dependent_var]

# Configuración del diseño de subgráficos
num_vars = len(independent_vars)
cols = 4  # Número de columnas en el diseño ajustadas automaticamente
rows = (num_vars + cols - 1) // cols  # Calcular el número de filas necesarias

# Crear los subgráficos
fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
axes = axes.flatten()  

# Generar los gráficos
for i, var in enumerate(independent_vars):
    ax = axes[i]
    ax.scatter(data[var], data[dependent_var], alpha=0.5)
    ax.set_title(f'Relación entre {var} y {dependent_var}')
    ax.set_xlabel(var)
    ax.set_ylabel(dependent_var)
    ax.grid()

# Ocultar los subgráficos vacíos si hay menos variables que subgráficos
for j in range(len(independent_vars), len(axes)):
    fig.delaxes(axes[j])
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Configuración del diseño para histogramas
variables = data.columns  # Todas las variables en el dataset
num_vars = len(variables)
cols = 4  # Número de columnas
rows = (num_vars + cols - 1) // cols  # Calcular el número de filas necesarias

# Crear los subgráficos
fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
axes = axes.flatten()  

# Generar histogramas para cada variable
for i, var in enumerate(variables):
    ax = axes[i]
    ax.hist(data[var], bins=50, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribución de {var}')
    ax.set_xlabel(var)
    ax.set_ylabel('Frecuencia')
    ax.grid()

# Ocultar los subgráficos vacíos si hay menos variables que subgráficos
for j in range(len(variables), len(axes)):
    fig.delaxes(axes[j])
plt.tight_layout()
plt.show()

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Verificar si hay GPU disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando dispositivo: {device}')

# Dividir los datos en variables independientes (X) y dependiente (y)
X = data.drop('median_house_value', axis=1)  # Variables independientes
y = data['median_house_value']  # Variable dependiente

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar las características
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Definir el modelo de red neuronal
class RegressionNN(nn.Module):
    def __init__(self, input_dim):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Instanciar el modelo
model = RegressionNN(X_train_tensor.shape[1]).to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Entrenar el modelo
epochs = 20
batch_size = 32

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train_tensor.size()[0])
    
    for i in range(0, X_train_tensor.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Usando dispositivo: cpu
Epoch [1/20], Loss: 11905438720.0000
Epoch [2/20], Loss: 13461288960.0000
Epoch [3/20], Loss: 3678214400.0000
Epoch [4/20], Loss: 3854074880.0000
Epoch [5/20], Loss: 8146366464.0000
Epoch [6/20], Loss: 4320578560.0000
Epoch [7/20], Loss: 2128161536.0000
Epoch [8/20], Loss: 10672419840.0000
Epoch [9/20], Loss: 6063344640.0000
Epoch [10/20], Loss: 3243199744.0000
Epoch [11/20], Loss: 1964498176.0000
Epoch [12/20], Loss: 6361620480.0000
Epoch [13/20], Loss: 3543914496.0000
Epoch [14/20], Loss: 2422369024.0000
Epoch [15/20], Loss: 7544202240.0000
Epoch [16/20], Loss: 5520684544.0000
Epoch [17/20], Loss: 4427939840.0000
Epoch [18/20], Loss: 2224330240.0000
Epoch [19/20], Loss: 4227188736.0000
Epoch [20/20], Loss: 1928957312.0000


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Hacer predicciones con el modelo en modo evaluación
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    
# Pasar de tensores a numpy arrays para calcular métricas
y_pred = y_pred_tensor.cpu().numpy()
y_true = y_test_tensor.cpu().numpy()

# Calcular métricas
mse = mean_squared_error(y_true, y_pred)
rmse = mse ** 0.5  # Raíz cuadrada del MSE
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# Mostrar métricas
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.2f}")



In [ ]:
import pandas as pd

# Variable para definir cuántas predicciones mostrar
num_predicciones = 20  # Modifica este valor manualmente según lo que desees visualizar

# Crear un DataFrame para comparar valores reales y predicciones
resultados = pd.DataFrame({
    'Valor Real': y_test[:num_predicciones].values,
    'Predicción': y_pred[:num_predicciones].flatten()
})

# Mostrar el DataFrame
print(resultados)

In [ ]:
import matplotlib.pyplot as plt

# Variable para definir cuántas predicciones mostrar
num_predicciones = 20  # Modifica este valor según lo que desees visualizar

# Obtener las primeras 'num_predicciones' predicciones y valores reales
y_real = y_test[:num_predicciones].values
y_predicciones = y_pred[:num_predicciones].flatten()  # Asegurar que las predicciones tengan una dimensión adecuada

# Crear el gráfico
plt.figure(figsize=(10, 6))
plt.plot(range(num_predicciones), y_real, label='Valores Reales', marker='o')
plt.plot(range(num_predicciones), y_predicciones, label='Predicciones', marker='x')
plt.title(f'Comparación de Valores Reales vs Predicciones ({num_predicciones} muestras)')
plt.xlabel('Índice')
plt.ylabel('Valores')
plt.legend()
plt.grid()
plt.show()


### Ahora ustedes deben ajustar las metricas del modelo... 
### Algunas sugerencias para que tomen en consideracion:
* Modificar la arquitectura de la red neuronal, como por ejemplo agregar capas, agregar capas de regularizacion como l2 o dropout.
* Utilizar una tasa de aprendizaje y modificar el optimizador
* Aumentar el numero de epocas
* Eliminar Outliers